## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import pandas as pd

task = 'FoodLightWhite'

saline_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodLight' AND health = 'saline' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(saline_q, conn)
saline_id = df['id'].to_list()
# IDs to remove
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]
# Filter them out
saline_id = [i for i in saline_id if i not in bad_id]

ghrelin_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodLight' AND health = 'ghrelin' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(ghrelin_q, conn)
ghrelin_id = df['id'].to_list()
# Filter them out
ghrelin_id = [i for i in ghrelin_id if i not in bad_id]

Exc_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodLightExcitatory' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(Exc_q, conn)
Exc_id = df['id'].to_list()
Exc_id = [i for i in Exc_id if i not in bad_id]

Inh_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodLightInhibitory' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(Inh_q, conn)
Inh_id = df['id'].to_list()
Inh_id = [i for i in Inh_id if i not in bad_id]

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"Exc_id: {Exc_id}\n")
print(f"Inh_id: {Inh_id}\n")


## Calculate entropy

In [ ]:
import importlib
import Python_scripts.Feature_functions.spatial_entropy
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.spatial_entropy)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.spatial_entropy import (
    get_spatial_entropy, get_batch_spatial_entropy_df
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


In [ ]:
r_saline = get_batch_spatial_entropy_df(saline_id, conn, radius_limit=0.7, max_time=300)
r_ghrelin = get_batch_spatial_entropy_df(ghrelin_id, conn, radius_limit=0.7, max_time=300)

# Add group labels
r_saline['group'] = 'Saline'
r_ghrelin['group'] = 'Ghrelin'

# Combine into one DataFrame
df_all = pd.concat([r_saline, r_ghrelin], ignore_index=True)

# Plot
plot_groupwise_bar(df_all, y='spatial_entropy', title='Spatial Entropy (r ≤ 0.7)', ylabel='Spatial Entropy')


### Batch call with different max_time

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_entropy_all_time.pdf'
with PdfPages(pdf_path) as pdf:
    for max_time in range(100, 1300, 100):
        print(f"[INFO] Analyzing max_time = {max_time} seconds")

        r_saline = get_batch_spatial_entropy_df(saline_id, conn, max_time=max_time)
        r_ghrelin = get_batch_spatial_entropy_df(ghrelin_id, conn, max_time=max_time)
        r_exc = get_batch_spatial_entropy_df(Exc_id, conn, max_time=max_time)
        r_inh = get_batch_spatial_entropy_df(Inh_id, conn, max_time=max_time)


        # Add group labels
        r_saline['group'] = 'Saline'
        r_ghrelin['group'] = 'Ghrelin'
        r_exc['group'] = 'Excitatory'
        r_inh['group'] = 'Inhibitory'
        
        # Combine into one DataFrame
        df_all = pd.concat([r_saline, r_ghrelin, r_exc, r_inh], ignore_index=True)

        # Plot
        fig, ax = plot_groupwise_bar(df_all, y='spatial_entropy', ylabel='Spatial Entropy')

        ax.set_title(f"{task} | max_time = {max_time}s")
        pdf.savefig(fig)
        plt.close(fig)  # Prevents inline display in notebooks

print(f"[DONE] All plots saved to: {pdf_path}")


### Batch call with different radius_limit

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_entropy_all_radius.pdf'
with PdfPages(pdf_path) as pdf:
    for radius in np.arange(0.1, 1.7, 0.2):
        print(f"[INFO] Analyzing radius = {radius:.1f}")

        r_saline = get_batch_spatial_entropy_df(saline_id, conn, radius_limit = radius, max_time=None)
        r_ghrelin = get_batch_spatial_entropy_df(ghrelin_id, conn, radius_limit = radius, max_time=None)

        # Add group labels
        r_saline['group'] = 'Saline'
        r_ghrelin['group'] = 'Ghrelin'
        
        # Combine into one DataFrame
        df_all = pd.concat([r_saline, r_ghrelin], ignore_index=True)

        # Plot
        fig, ax = plot_groupwise_bar(df_all, y='spatial_entropy', ylabel='Spatial Entropy')

        ax.set_title(f"{task} | radius = {radius:.1f}")
        pdf.savefig(fig)
        plt.close(fig)  # Prevents inline display in notebooks

print(f"[DONE] All plots saved to: {pdf_path}")


### Batch call with different radius and time limit

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_radial_cdf_radius_time_grid.pdf'
with PdfPages(pdf_path) as pdf:
    for max_time in range(100, 1300, 300):
        for radius in np.arange(0.1, 1.7, 0.3):
            print(f"[INFO] Analyzing radius = {radius:.1f}, max_time = {max_time}")

            r_saline = get_batch_radial_distances(saline_id, conn, radius_limit=radius, max_time=max_time)
            r_ghrelin = get_batch_radial_distances(ghrelin_id, conn, radius_limit=radius, max_time=max_time)

            if len(r_saline) == 0 or len(r_ghrelin) == 0:
                print(f"[WARNING] Skipping radius = {radius:.1f}, max_time = {max_time} due to missing data")
                continue

            r_saline_all = np.concatenate(r_saline)
            r_ghrelin_all = np.concatenate(r_ghrelin)

            fig, ax = compare_radial_distributions(
                r_saline_all, r_ghrelin_all,
                kind='cdf', return_data=False
            )

            ax.set_title(f"{task} | radius = {radius:.1f}, max_time = {max_time}")
            pdf.savefig(fig)
            plt.close(fig)

print(f"[DONE] All plots saved to: {pdf_path}")


### Sweep grid_size with Fixed radius=0.7 and max_time=500

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# Output PDF name
pdf_path = f'{task}_entropy_grid_test.pdf'

# Fixed parameters
radius = 0.7
max_time = 500

with PdfPages(pdf_path) as pdf:
    for grid_size in range(5, 30, 5):  # Try 5x5 to 25x25
        print(f"[INFO] Analyzing grid_size = {grid_size}x{grid_size}")

        r_saline = get_batch_spatial_entropy_df(
            saline_id, conn,
            max_time=max_time, radius_limit=radius, grid_size=grid_size
        )
        r_ghrelin = get_batch_spatial_entropy_df(
            ghrelin_id, conn,
            max_time=max_time, radius_limit=radius, grid_size=grid_size
        )

        # Add group labels
        r_saline['group'] = 'Saline'
        r_ghrelin['group'] = 'Ghrelin'

        # Combine into one DataFrame
        df_all = pd.concat([r_saline, r_ghrelin], ignore_index=True)

        # Plot
        fig, ax = plot_groupwise_bar(df_all, y='spatial_entropy',
                                     ylabel='Spatial Entropy',
                                     plot_type='box', order=['Saline', 'Ghrelin'], show_stats=True)

        ax.set_title(f"{task} | grid = {grid_size}x{grid_size}")
        pdf.savefig(fig)
        plt.close(fig)

print(f"[DONE] Grid size sweep saved to: {pdf_path}")


## Entropy distribution

In [ ]:
import importlib
import Python_scripts.Feature_functions.entropy_by_distance_to_center
import Python_scripts.Data_analysis.plot_feature_vs_binned_distance

importlib.reload(Python_scripts.Feature_functions.entropy_by_distance_to_center)
importlib.reload(Python_scripts.Data_analysis.plot_feature_vs_binned_distance)

from Python_scripts.Feature_functions.entropy_by_distance_to_center import (
    get_entropy_by_distance_to_center, get_batch_entropy_by_distance_to_center
)
from Python_scripts.Data_analysis.plot_feature_vs_binned_distance import plot_feature_vs_binned_distance


In [ ]:
import numpy as np
import pandas as pd

df_saline = get_batch_entropy_by_distance_to_center(
    id_list=saline_id, conn=conn, center=None, max_time=500,
    normalize=False, group_label='Saline', verbose=False
)

df_ghrelin = get_batch_entropy_by_distance_to_center(
    id_list=ghrelin_id, conn=conn, center=None, max_time=500,
    normalize=False, group_label='Ghrelin', verbose=False
)

df_all = pd.concat([df_saline, df_ghrelin], ignore_index=True)


In [ ]:
stats_df, fig, ax = plot_feature_vs_binned_distance(
    df_all,
    feature_col='entropy',
    ylabel='Entropy'
)

ax.set_title(f"{task}")
fig.savefig(f"{task}_binned_entropy.pdf", dpi=300)

# Save stats_df as CSV
stats_df.to_csv(f"{task}_binned_entropy_stats.csv", index=False)

print(stats_df.head(n=2))

### Batch call with different max_time

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Define output file and range
pdf_path = f"{task}_binned_entropy_by_time.pdf"
max_time_array = np.arange(100, 1300, 200)  # or any array you define

with PdfPages(pdf_path) as pdf:
    for max_time in max_time_array:
        print(f"[INFO] Running for max_time = {max_time}s")

        df_saline = get_batch_entropy_by_distance_to_center(
            id_list=saline_id, conn=conn, center=None,
            max_time=max_time, normalize=False,
            group_label='Saline', verbose=False
        )

        df_ghrelin = get_batch_entropy_by_distance_to_center(
            id_list=ghrelin_id, conn=conn, center=None,
            max_time=max_time, normalize=False,
            group_label='Ghrelin', verbose=False
        )

        df_all = pd.concat([df_saline, df_ghrelin], ignore_index=True)
        df_all['group'] = pd.Categorical(df_all['group'], categories=['Saline', 'Ghrelin'], ordered=True)

        stats_df, fig, ax = plot_feature_vs_binned_distance(
            df_all, feature_col='entropy',
            ylabel='Entropy', title=f"{task} | max_time = {max_time}s"
        )

        pdf.savefig(fig)
        plt.close(fig)

print(f"[DONE] All plots saved to: {pdf_path}")


### Save outputs

In [ ]:
import numpy as np
import pandas as pd

heights, bin_edges = np.histogram(r_saline_all, bins=100, density=True)
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

df = pd.DataFrame({'bin_center': bin_centers, 'height': heights})
df.to_csv(f'{task}_radial_histogram_saline.csv', index=False)

heights, bin_edges = np.histogram(r_ghrelin_all, bins=100, density=True)
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

df = pd.DataFrame({'bin_center': bin_centers, 'height': heights})
df.to_csv(f'{task}_radial_histogram_ghrelin.csv', index=False)


In [ ]:
import importlib
import Python_scripts.Feature_functions.entropy_by_distance_to_center

importlib.reload(Python_scripts.Feature_functions.entropy_by_distance_to_center)

from Python_scripts.Feature_functions.entropy_by_distance_to_center import (
    get_entropy_by_distance_to_center, get_batch_entropy_by_distance_to_center
)


In [ ]:
import numpy as np

df_saline = get_batch_entropy_by_distance_to_center(
    id_list=saline_id, conn=conn, center=None,
    bins=np.arange(0, 1.1, 0.1),
    max_time=500, group_label='Saline', verbose=True
)
